In [1]:
import tensorflow as tf
from sklearn.neighbors import NearestNeighbors
import pandas as pd
import numpy as np

In [2]:
hotel = pd.read_json("./hotel_add.json")
restaurant = pd.read_json("./restaurant_add.json")
todo = pd.read_json("./todo_add.json")

In [20]:
def recommend_places(activity, category, latitude, longitude, quantity):
    # Get the user's preferences from the request

    # Define the user's preferences
    activity = str(activity)
    category = str(category)
    latitude = latitude
    longitude = longitude

    # Calculate the distance between the user's location and the places' locations
    todo["distance"] = tf.sqrt(
        tf.square(todo["latitude"] - latitude) + tf.square(todo["longitude"] - longitude)
    )

    # Filter the places based on the user's preferences
    filtered_places = todo[
        (todo["activity"] == activity) & (todo["category"] == category)
    ]

    # Check if there are any matching places
    if filtered_places.empty:
        return {"recommendations": []}

    # Perform collaborative filtering using k-nearest neighbors
    X = filtered_places[["latitude", "longitude", "rating", "distance"]].values
    nbrs = NearestNeighbors(n_neighbors=quantity).fit(X)
    distances, indices = nbrs.kneighbors([[latitude, longitude, 5, 0]])

    # Get the top recommendations based on nearest neighbors
    top_indices = indices[0]
    top_recommendations = filtered_places.iloc[top_indices]

    # Create a response dictionary with the recommendations
    recommendations = []
    for _, row in top_recommendations.iterrows():
        recommendation = row.to_dict()
        recommendations.append(recommendation)

    return {"place_recommendation": recommendations}

In [21]:
# Function to recommend restaurants based on latitude and longitude
def recommend_restaurants(latitude, longitude, quantity):

     # Calculate the distance between the user's location and the places' locations
     restaurant["distance"] = tf.sqrt(
         tf.square(restaurant["latitude"] - latitude) + tf.square(restaurant["longitude"] - longitude)
     )

     # Takes latitude, longitude, rating, and distance columns as features
     X = restaurant[['latitude', 'longitude', 'rating', 'distance']].values

     # Build the KNN model with the number of neighbors (k) = quantity
     knn = NearestNeighbors(n_neighbors=quantity)
     knn.fit(X)

     # Search for 5 nearest neighbors of given position
     distances, indices = knn.kneighbors([[latitude, longitude, 5, 0]])

     # Displays recommended restaurants
     restaurant_recommendations = []
     for index in indices[0]:
         recommendation = restaurant.iloc[index].to_dict()
         restaurant_recommendations. append(recommendation)

     # Checks if distance is a multiple of 20, then sorts by best rating
     if int(restaurant.iloc[indices[0][0]]['distance']) % 20 == 0:
         restaurant_recommendations = sorted(restaurant_recommendations, key=lambda x: x['rating'], reverse=True)

     return {"restaurant_recommendation": restaurant_recommendations}

In [23]:
# Function to recommend restaurants based on latitude and longitude
def recommend_hotels(latitude, longitude, quantity):

     # Calculate the distance between the user's location and the places' locations
     hotel["distance"] = tf.sqrt(
         tf.square(hotel["latitude"] - latitude) + tf.square(hotel["longitude"] - longitude)
     )

     # Takes latitude, longitude, rating, and distance columns as features
     X = hotel[['latitude', 'longitude', 'hotel_class', 'rating', 'distance']].values

     # Build the KNN model with the number of neighbors (k) = quantity
     knn = NearestNeighbors(n_neighbors=quantity)
     knn.fit(X)

     # Search for 5 nearest neighbors of given position
     distances, indices = knn.kneighbors([[latitude, longitude, 5, 5, 0]])

     # Displays recommended restaurants
     recommendations = []
     for index in indices[0]:
         recommendation = hotel.iloc[index].to_dict()
         recommendations. append(recommendation)

     # Checks if distance is a multiple of 20, then sorts by best rating
     if int(hotel.iloc[indices[0][0]]['distance']) % 20 == 0:
         recommendations = sorted(recommendations, key=lambda x: x['rating'], reverse=True)

     return {"hotel_recommendation": recommendations}

In [52]:
def itenerary(activity, category, latitude, longitude):
  todo_recommendation = recommend_places(activity, category, latitude, longitude, 1)

  if len(todo_recommendation) > 0:
      todo_latitude = todo_recommendation['place_recommendation'][0]['latitude']
      todo_longitude = todo_recommendation['place_recommendation'][0]['longitude']

      hotel_recommendation = recommend_hotels(todo_latitude, todo_longitude, 3)
      restaurant_recommendation = recommend_restaurants(todo_latitude, todo_longitude, 3)

      return {
          'todo_recommendation': todo_recommendation,
          'hotel_recommendation': hotel_recommendation,
          'restaurant_recommendation': restaurant_recommendation
      }

  else:
      return {
          'todo_recommendation':[],
          'hotel_recommendation':[],
          'restaurant_recommendation':[]
      }



In [12]:
# Example usage
user_activity = "Outdoor"
user_category = "Beach"
user_latitude = -1.2654
user_longitude = 116.8311

In [53]:
itenerary(user_activity, user_category, user_latitude, user_longitude)

{'todo_recommendation': {'place_recommendation': [{'location_id': 25304538,
    'place_name': 'Watu Beach Lamaru',
    'city': 'Balikpapan',
    'rating': 4.5,
    'latitude': -1.19705,
    'longitude': 117.00225,
    'opening_hours': {'week_ranges': [[{'open_time': 480, 'close_time': 1020}],
      [{'open_time': 480, 'close_time': 1020}],
      [{'open_time': 480, 'close_time': 1020}],
      [{'open_time': 480, 'close_time': 1020}],
      [{'open_time': 480, 'close_time': 1020}],
      [{'open_time': 480, 'close_time': 1020}],
      [{'open_time': 480, 'close_time': 1020}]],
     'timezone': 'Asia/Makassar'},
    'contact_number': '+62 819-2882-3224',
    'photo': None,
    'address': 'Jl. Mulawarman Rt 16, Balikpapan 76118 Indonesia',
    'min_price': 20000,
    'max_price': 50000,
    'activity': 'Outdoor',
    'category': 'Beach',
    'description': 'Watu Beach Lamaru',
    'distance': 0.18429336667389598}]},
 'hotel_recommendation': {'hotel_recommendation': [{'original_id': 175962